CP 321 Final Project

By: Aryan Amin (193265300) and Nicolas Lozano (150701040)

This is our project for the CP 321 final assignment. We are working on the dataset that is providing the information about the key infrastructure projects that is either completed or that are currenlty in progress in the state of Ontario. The dataset is taken from the website of government of Canada. The website link is https://search.open.canada.ca/en/od/?sort=score%20desc&page=1&search_text=key%20infrastructure&od-search-format=CSV. We followed the ASSERT model that we learned in our course to complete this project.

The question we are trying to answer is 